In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, BatchNormalization, Activation, Dropout
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import backend as K
from tqdm import tqdm
from glob import glob
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import math

In [3]:
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 3817
nb_validation_samples = 933
epochs = 150
batch_size = 8

In [4]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [5]:
# Data import
train_datagen = ImageDataGenerator(rotation_range=25,
                                   rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,  # this is the target directory
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,  # this is the target directory
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 3817 images belonging to 12 classes.
Found 933 images belonging to 12 classes.


In [6]:
# Model definition
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(12, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [7]:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
# Training
checkpointer = ModelCheckpoint(filepath='weights_gpu_v4.hdf5',
                               verbose=1, save_best_only=True)

history = model.fit_generator(train_generator,
        steps_per_epoch=int(math.ceil(nb_train_samples / batch_size)),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=int(math.ceil(nb_validation_samples / batch_size)),
        verbose=1,
        callbacks=[checkpointer])

In [8]:
model.load_weights('weights_gpu_v4.hdf5')

seedlings_names = [item[11:] for item in sorted(glob("data/train/*"))]

f = open('results_v4.csv', 'w')
f.write('file,species\n')

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(img_width, img_height))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

test_files = glob("data/test/*")
test = paths_to_tensor(test_files).astype('float32')/255

output = model.predict(test)
for [o, name] in zip(output, test_files):
    f.write(name[10:] + ',')
    seedling = seedlings_names[np.argmax(o)]
    f.write(seedling)
    f.write('\n')
f.close()

100%|██████████| 794/794 [00:13<00:00, 57.36it/s]
